# Questions on Catch the Phase.
1. State the motivation behind the quantum phase estimation (QPE) subroutine. \
Answer: QPE is used to solve linear system of equations and finding the ground energy if a molecule. 

2. List some applications of QPE. \
Answer: Mentioned in 1. 

3. Describe the phase kickback technique. \
Answer: Let us assume we have an controlled unitary operator $U$ and its corresponding eigenval and eigen vec $\lambda$ and $\ket{\psi_{\lambda}}$. We can prepare the target wires in $U$ eigen-vector states. The effect of applying $U$ on $\ket{\psi_\lambda}$ is defined as 
$$
U \ket{\psi_\lambda} = \lambda \ket{\psi_\lambda} = e^{2i \pi \theta } \ket{\psi_\lambda}
$$ 
the eigenvalue corresponding to this eigenstate is kicked back to the control wires. Then, the control wires pick up a phase of $e^{2i \pi \theta }$. This is called phase kickback.

4. Estimate a phase using phase kickback. \
Answer: The phase via kickback is $e^{2i \pi \theta }$.

In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
Z_gate = np.asarray([[1,1],[1,-1]])
eigen_value_z,eigen_vec_z = np.linalg.eig(Z_gate)
print(f"The Eigen values are: {eigen_value_z}")
print(f"The eigen vectors are:{eigen_vec_z}")

The Eigen values are: [ 1.41421356 -1.41421356]
The eigen vectors are:[[ 0.92387953 -0.38268343]
 [ 0.38268343  0.92387953]]


In [3]:
lamda_1_z,lamda_2_z = eigen_value_z
psi_1_z,psi_2_z = eigen_vec_z

Since the eigenvectors form an orthonormal set, all unitary matrices can be written in a diagonal representation or an orthonormal decomposition (or equivalently, a spectral decomposition).
$
\begin{equation}
U = \sum_{\lambda} e^{2i \pi \theta} \ket{\psi_{\lambda}} \bra{\psi_{\lambda}}
\end{equation}
$
where $e^{2i \pi \theta}$ is an eigenvalue of $\ket{ \psi_{\lambda} }$

In [4]:
U = 0
for i in range(len(eigen_value_z)):
    U += eigen_value_z[i]*np.kron(eigen_vec_z[i],eigen_vec_z[i])
print(f"The unitary matrix is: {U}")


The unitary matrix is: [ 1. -1. -1. -1.]


Exercise P.1.3. Write the Tgate and the X gate in terms of their eigenbases.

In [5]:
X_gate = np.array([[0,1],[1,0]])
T_gate = np.asarray([[1,0],[0,np.exp(1j*np.pi/4)]])
eigen_val_x,eigen_vec_x = np.linalg.eigh(X_gate)
eigen_val_t,eigen_vec_t = np.linalg.eigh(T_gate)
print(f"The Eigen values of X gate are: {eigen_val_x}")
print(f"The eigen vectors of X gate are:{eigen_vec_x}")
print()
print(f"The Eigen values of T gate are: {eigen_val_t}")
print(f"The eigen vectors of T gate are:{eigen_vec_t}")

The Eigen values of X gate are: [-1.  1.]
The eigen vectors of X gate are:[[-0.70710678  0.70710678]
 [ 0.70710678  0.70710678]]

The Eigen values of T gate are: [0.70710678 1.        ]
The eigen vectors of T gate are:[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]


In [6]:
U_x,U_t = 0,0
for i in range(len(eigen_val_x)):
    U_x += eigen_val_x[i]*np.kron(eigen_vec_x[i],eigen_vec_x[i])
    U_t += eigen_val_t[i]*np.kron(eigen_vec_t[i],eigen_vec_t[i])

print(f"The unitary matrix of X gate is: {U_x}")
print()
print(f"The unitary matrix of T gate is: {U_t}")

The unitary matrix of X gate is: [0. 1. 1. 0.]

The unitary matrix of T gate is: [1.        +0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [7]:
CNOT_gate = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
Toffoli_gate = np.identity(8)
Toffoli_gate[-1,-1],Toffoli_gate[-2,-2], = 0,0
Toffoli_gate[-1,-2],Toffoli_gate[-2,-1], = 1,1
eigen_val_cnot,eigen_vec_cnot = np.linalg.eigh(CNOT_gate)
eigen_val_toff,eigen_vec_toff = np.linalg.eigh(Toffoli_gate)

prove that the probability of observing 0 when measuring the first qubit is ($1 - sin^2 \pi \theta$) and  and the probability of observing 1 is $sin^2 \pi \theta$ \
The circuit is $H-CU-H (\ket{0} \otimes \ket{\psi})$ where $\ket{\psi}$ is an eigen vector if controlled unitary operator $CU$. 

Applying the given gate in orders, we will be at state,
$$
\ket{\Phi} = \frac{1}{2}[(1 + e^{2 i \pi \theta}) \ket{0}  + (1 - e^{2 i \pi \theta}) \ket{1} ] \otimes \ket{\psi} \\

Therefore, P(\ket{0}) = \frac{(1 + e^{2 i \pi \theta})}{2^2}^2 = 1-sin^2 \pi \theta \\

and, P(\ket{1}) = sin^2 \pi \theta
$$

### Codercise P.1.1.
 You are given a unitary that is promised to be either the Z gate or the −Z gate. Write a quantum program using phase kickback that will result in the state |0⟩ with a probability of 100% on the first qubit if Z is applied and |1⟩ with a probability of 100% on the first qubit if −Z is applied.

In [9]:
dev = qml.device("default.qubit", wires=2, shots=1)

@qml.qnode(dev)
def guess_the_unitary(unitary):
    """Given a unitary that performs a Z or a -Z operation
    on a qubit, guess which one it is.
    
    Args: 
        U (array[complex]): A unitary matrix, guaranteed to be either Z or -Z.
    
    Returns:
        array [int]:  Probabilities on  on the first qubit
        using qml.probs()
    """
    ##################
    # YOUR CODE HERE #
    ##################  
    qml.Hadamard(wires = 0)
    qml.ControlledQubitUnitary(unitary, control_wires = 0, wires = 1)
    qml.Hadamard(wires = 1)
    return qml.probs(wires = [0])
    

# Z gate 
U = qml.PauliZ.matrix 

# -Z gate
# U = (-1)*qml.PauliZ.matrix

# print(guess_the_unitary(U))


### Codercise P.1.2. 
Find the eigenvalues of the X gate

In [11]:
dev = qml.device("default.qubit", wires=2)
        
@qml.qnode(dev)
def phase_kickback_X(eigenvector):
    """ Given an eigenvector of X, 
    apply the phase kickback circuit to observe 
    the probabilities on the control wire
    
    Args: 
        eigenvector(String): A string "plus" or "minus" depicting 
        the eigenvector of X
    
    Returns:
        array[int]: Measurement outcome on the first qubit using qml.probs()
    """
    # Prepare |ψ>
    ##################
    # YOUR CODE HERE #
    ##################  
    if eigenvector == "minus":  
        qml.PauliX (wires = 1)
    qml.Hadamard(wires = 1)
    qml.Hadamard(wires = 0)
    qml.CNOT(wires = [0,1])
    qml.Hadamard(wires = 0)

    # Phase kickback
    ##################
    # YOUR CODE HERE #
    ################## 
 
    return qml.probs(wires=[0])   

print(phase_kickback_X("plus"))
print(phase_kickback_X("minus"))

# MODIFY EIGENVALUES BELOW 
eigenvalue_of_X_plus = 1
eigenvalue_of_X_minus = -1


[1. 0.]
[0. 1.]
